In [ ]:
# Download and install the webdriver at first, the website is as follows:
#     https://sites.google.com/a/chromium.org/chromedriver/

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from tqdm import tqdm
import dateutil

m = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [2]:
data = []

for i in range(11, 12):
    for j in range(m[i]):
        
        print('2013-' + str(i+1) + '-' + str(j+1))
        browser = webdriver.Chrome()
        browser.get('https://www.wunderground.com/history/daily/gb/london/EGLC/date/2013-' + str(i+1) + '-' + str(j+1))
        
        sleep(10)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        table = soup.find_all('table')[2].find_all('tr')
        
        n = len(table)
        for k in range(1, n):
            tmp = ['2013-%02d-%02d' %(i+1, j+1), table[k].find_all('span')[0].get_text(), table[k].find_all(class_='wu-value wu-value-to')[0].get_text()]
            data.append(tmp)
        
        browser.quit()

2013-12-1
2013-12-2
2013-12-3
2013-12-4
2013-12-5
2013-12-6
2013-12-7
2013-12-8
2013-12-9
2013-12-10
2013-12-11
2013-12-12
2013-12-13
2013-12-14
2013-12-15
2013-12-16
2013-12-17
2013-12-18
2013-12-19
2013-12-20
2013-12-21
2013-12-22
2013-12-23
2013-12-24
2013-12-25
2013-12-26
2013-12-27
2013-12-28
2013-12-29
2013-12-30
2013-12-31


In [36]:
data = pd.DataFrame(data)
data.columns = ['Date', 'Time', 'T']
data['Date'] = data['Date'].astype('datetime64[ns]')
data.to_csv('../data/weather/weather_London_12.csv', index=None)

## Clean weather data (Cork)

In [14]:
data = pd.read_csv('../data/weather/weather_Cork.csv')
data['Date'] = data['Date'].astype('datetime64[ns]')

In [16]:
for i in tqdm(range(12)):
    for j in range(m[i]):
        tmp = data[data['Date'] == '2010-%02d-%02d' % (i+1, j+1)]['Time']
        
        if 'P' in tmp.values[0]:
            k = 0
            while 'P' in tmp.values[k]:
                data['Date'][tmp.index[k]] = data['Date'][tmp.index[k]] - dateutil.relativedelta.relativedelta(days=1)
                k = k+1

100%|██████████| 12/12 [00:00<00:00, 54.79it/s]


In [261]:
# Datetime
datetime = []
for i in range(len(data)):
    datetime.append(pd.to_datetime(str(data['Date'][i]) + ' ' + data['Time'][i]))
    
data = pd.concat([data, pd.DataFrame(datetime, columns=['DateTime'])], axis=1)

In [262]:
# Transfer to DateTime
data = data[['DateTime', 'T']]
data.to_csv('../data/weather/weather_Cork_cleaned.csv', index=None)
data = pd.read_csv('../data/weather/weather_Cork_cleaned.csv')

In [263]:
# Delete rows with T == 0
for i in range(len(data)):
    if data['T'][i] == 0:
        data.drop(index=[i], inplace=True)

In [264]:
data['Miss'] = 0

In [265]:
# Construct the empty DataFrame to store the interpolation
add = pd.DataFrame(columns=['DateTime', 'T', 'Miss'])

for j in range(1, 365*48):
    t = pd.to_datetime('2010-01-01 00:00:00') + dateutil.relativedelta.relativedelta(minutes=30*j)
    if str(t) not in data['DateTime'].values:

        # Use average values as interpolation
        value = (data[data['DateTime'] > str(t)].iloc[0, 1] + data[data['DateTime'] < str(t)].iloc[-1, 1])/2
        tmp = pd.DataFrame([[str(t), value, 1]], columns=['DateTime', 'T', 'Miss'])
        add = pd.concat([add, tmp], axis=0)

In [266]:
data = pd.concat([data, add]).sort_values(by='DateTime')
data.reset_index(inplace=True, drop=True)
data.to_csv('../data/weather/weather_Cork_cleaned.csv', index=None)

## Clean weather data (Dublin)

In [19]:
data = pd.read_csv('../data/weather/weather_Dublin.csv')
data['Date'] = data['Date'].astype('datetime64[ns]')

In [20]:
for i in range(12):
    for j in range(m[i]):
        tmp = data[data['Date'] == '2010-%02d-%02d' % (i+1, j+1)]['Time']
        
        if 'P' in tmp.values[0]:
            k = 0
            while 'P' in tmp.values[k]:
                data['Date'][tmp.index[k]] = data['Date'][tmp.index[k]] - dateutil.relativedelta.relativedelta(days=1)
                k = k+1

/Users/kaishuai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
# Datetime
datetime = []
for i in range(len(data)):
    datetime.append(pd.to_datetime(str(data['Date'][i]) + ' ' + data['Time'][i]))
    
data = pd.concat([data, pd.DataFrame(datetime, columns=['DateTime'])], axis=1)

In [80]:
# Transfer to DateTime
data = data[['DateTime', 'T']]
data.to_csv('../data/weather/weather_Dublin_cleaned.csv', index=None)
data = pd.read_csv('../data/weather/weather_Dublin_cleaned.csv')

In [81]:
# Delete rows with T == 0
for i in range(len(data)):
    if data['T'][i] == 0:
        data.drop(index=[i], inplace=True)

In [82]:
data['Miss'] = 0

In [83]:
# Construct the empty DataFrame to store the interpolation
add = pd.DataFrame(columns=['DateTime', 'T', 'Miss'])

for j in range(1, 365*48):
    t = pd.to_datetime('2010-01-01 00:00:00') + dateutil.relativedelta.relativedelta(minutes=30*j)
    if str(t) not in data['DateTime'].values:

        # Use average values as interpolation
        value = (data[data['DateTime'] > str(t)].iloc[0, 1] + data[data['DateTime'] < str(t)].iloc[-1, 1])/2
        tmp = pd.DataFrame([[str(t), value, 1]], columns=['DateTime', 'T', 'Miss'])
        add = pd.concat([add, tmp], axis=0)

In [84]:
data = pd.concat([data, add]).sort_values(by='DateTime')
data.reset_index(inplace=True, drop=True)
data.to_csv('../data/weather/weather_Dublin_cleaned.csv', index=None)

In [85]:
# There are some outliers which make the total number is greater than 17520(48 * 365)
for i in range(12):
    for j in range(m[i]):
        tmp = data[(data['DateTime'] >= '2010-%02d-%02d 00:00:00' % (i+1, j+1)) & (data['DateTime'] < '2010-%02d-%02d 00:00:00' % (i+1, j+2))]
        if len(tmp) != 48:
            print(i+1, j+1, len(tmp))

5 22 49
8 2 49
11 30 49
12 6 49


In [95]:
i=12
j=6

In [96]:
tmp = data[(data['DateTime'] >= '2010-%02d-%02d 00:00:00' % (i, j)) & (data['DateTime'] < '2010-%02d-%02d 00:00:00' % (i, j+1))]

In [97]:
tmp

,DateTime,T,Miss
16275,2010-12-06 00:00:00,27.0,0
16276,2010-12-06 00:30:00,27.0,0
16277,2010-12-06 01:00:00,25.0,0
16278,2010-12-06 01:30:00,25.0,0
16279,2010-12-06 02:00:00,25.0,0
16280,2010-12-06 02:30:00,25.0,0
16281,2010-12-06 03:00:00,23.0,0
16282,2010-12-06 03:30:00,25.0,0
16283,2010-12-06 04:00:00,23.0,0
16284,2010-12-06 04:30:00,23.0,0


In [98]:
data.drop(index=[6779, 10246, 16033, 16293], inplace=True)

In [99]:
len(data)

17520

In [100]:
data.to_csv('../data/weather/weather_Dublin_cleaned.csv', index=None)

## Clean weather data (London)

In [23]:
data = pd.read_csv('../data/weather/weather_London.csv')
data['Date'] = data['Date'].astype('datetime64[ns]')

In [24]:
for i in range(12):
    for j in range(m[i]):
        tmp = data[data['Date'] == '2013-%02d-%02d' % (i+1, j+1)]['Time']
        
        if 'P' in tmp.values[0]:
            k = 0
            while 'P' in tmp.values[k]:
                data['Date'][tmp.index[k]] = data['Date'][tmp.index[k]] - dateutil.relativedelta.relativedelta(days=1)
                k = k+1

/Users/kaishuai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
# Datetime
datetime = []
for i in range(len(data)):
    datetime.append(pd.to_datetime(str(data['Date'][i]) + ' ' + data['Time'][i]))
    
data = pd.concat([data, pd.DataFrame(datetime, columns=['DateTime'])], axis=1)

In [128]:
# Transfer from [Date, Time] to [DateTime]
data = data[['DateTime', 'T']]
data.to_csv('../data/weather/weather_London_cleaned.csv', index=None)
data = pd.read_csv('../data/weather/weather_London_cleaned.csv')

In [129]:
# Delete rows with T == 0
for i in range(len(data)):
    if data['T'][i] == 0:
        data.drop(index=[i], inplace=True)

In [130]:
data['Miss'] = 0
data.drop(index=[0], inplace=True)

In [131]:
# Construct the empty DataFrame to store the interpolation
add = pd.DataFrame(columns=['DateTime', 'T', 'Miss'])

for j in range(1, 365*48-1):
    t = pd.to_datetime('2013-01-01 00:20:00') + dateutil.relativedelta.relativedelta(minutes=30*j)
    if str(t) not in data['DateTime'].values:

        # Use average values as interpolation
        value = (data[data['DateTime'] > str(t)].iloc[0, 1] + data[data['DateTime'] < str(t)].iloc[-1, 1])/2
        tmp = pd.DataFrame([[str(t), value, 1]], columns=['DateTime', 'T', 'Miss'])
        add = pd.concat([add, tmp], axis=0)

In [132]:
data = pd.concat([data, add]).sort_values(by='DateTime')
data.reset_index(inplace=True, drop=True)

In [134]:
# Add the last point
tmp = pd.DataFrame([['2013-12-31 23:50:00', 48, 0]], columns=['DateTime', 'T', 'Miss'])
data = pd.concat([data, tmp], axis=0)

In [136]:
data.to_csv('../data/weather/weather_London_2013_cleaned.csv', index=None)

## Combine Cork and Dublin

In [34]:
df1 = pd.read_csv('../data/weather/weather_Cork_cleaned.csv')
df2 = pd.read_csv('../data/weather/weather_Dublin_cleaned.csv')

In [35]:
# Detect continuous missing values
def detect_c_miss(df):
    c_miss = []
    for i in range(len(df)):
        if df['Miss'][i] == 0:
            c_miss.append(0)
        else:
            count = 1
            k = i
            while k-1>=0 and df['Miss'][k-1] == 1:
                count = count + 1
                k = k - 1

            k = i
            while k+1<len(df) and df['Miss'][k+1] == 1:
                count = count + 1
                k = k + 1

            if count > 2:
                c_miss.append(1)
            else:
                c_miss.append(0)
    return c_miss

In [36]:
df1 = pd.concat([df1, pd.DataFrame(detect_c_miss(df1), columns=['Continous Miss'])], axis=1)
df2 = pd.concat([df2, pd.DataFrame(detect_c_miss(df2), columns=['Continous Miss'])], axis=1)

In [37]:
data = pd.DataFrame(columns=['DateTime', 'T', 'Miss'])

for i in tqdm(range(len(df1))):
    miss = 1
    if df1['Continous Miss'][i] == 0 and df2['Continous Miss'][i] == 0:
        value = (df1['T'][i] + df2['T'][i]) / 2
        miss = 0
    elif df1['Continous Miss'][i] == 0 and df2['Continous Miss'][i] == 1:
        value = df1['T'][i]
    elif df1['Continous Miss'][i] == 1 and df2['Continous Miss'][i] == 0:
        value = df2['T'][i]
    else:
        value = (df1['T'][i] + df2['T'][i]) / 2
    tmp = pd.DataFrame([[df1['DateTime'][i], value, miss]], columns=['DateTime', 'T', 'Miss'])
    data = pd.concat([data, tmp], axis=0)

100%|██████████| 17520/17520 [00:43<00:00, 406.78it/s]


In [39]:
data.to_csv('../data/weather/weather_Irish_2010_cleaned.csv', index=None)